In [ ]:
!pip install catboost
!pip install deepchecks
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.3/380.3 kB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 10.2 MB/s eta 0:00:00
  Created wheel for PyNomaly: filename=PyNomaly-0.3.3-py3-none-any.whl 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from copy import deepcopy

from sklearn.model_selection import train_test_split

from catboost import CatBoostRegressor, CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, auc
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
plt.rcParams['font.size'] = 10
from deepchecks.tabular import Dataset
from deepchecks.tabular.checks import FeatureDrift
import lightgbm as lgbm

from tqdm.auto import tqdm

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.multiclass import unique_labels
from xgboost import XGBClassifier

In [ ]:
def weighted_mean_absolute_error(y_true, y_pred, weights):

    '''

    Weighted mean absolute error.

    Parameters
    ----------
    y_true: ndarray
        Ground truth
    y_pred: ndarray
        Array of predictions

    Returns
    -------
    rmsle: float
        Weighted mean absolute error

    References
    ----------
    .. [1] https://kaggle-metrics.readthedocs.io/en/latest/_modules/kaggle_metrics/regression.html

    '''

    return (weights * np.abs(y_true - y_pred)).mean()

In [ ]:
# def read_statistics_data():
#     '''
#     Чтение статистических данных:
#     средняя зарплата по регионам/месяцам - https://gogov.ru/articles/average-salary
#     величина прожиточного минимума - https://rosstat.gov.ru/vpm (Величина прожиточного минимума, установленная на 2024 год по субъектам Российской Федерации)
#     '''

#     salary_month_df = pd.read_csv('salary_month.csv', sep=";", decimal=",")
#     salary_month_df = salary_month_df.set_index('Область')
#     salary_month_df

#     vpm_df = pd.read_csv('vpm_2024.csv', sep=";", decimal=",")
#     vpm_df = vpm_df.set_index('Область')
#     vpm_df = vpm_df.rename(columns={"2024": "vpm_2024"})

#     return salary_month_df, vpm_df

In [ ]:
def add_statistics_data_to_df(df):
    '''
    Чтение статистических данных:
    средняя зарплата по регионам/месяцам - https://gogov.ru/articles/average-salary
    величина прожиточного минимума - https://rosstat.gov.ru/vpm (Величина прожиточного минимума, установленная на 2024 год по субъектам Российской Федерации)
    рейтинг регионов по зарплатам - https://ria.ru/20231120/zarplata-1910154567.html
    индекс стоимости жизни по регионам - https://rosstat.gov.ru/statistics/price
    '''
    salary_month_df = pd.read_csv('salary_month.csv', sep=";", decimal=",")
    salary_month_df = salary_month_df.set_index('Область')
    salary_month_df

    vpm_df = pd.read_csv('vpm_2024.csv', sep=";", decimal=",")
    vpm_df = vpm_df.set_index('Область')
    vpm_df = vpm_df.rename(columns={"2024": "vpm_2024"})

    salary_rating_df = pd.read_csv('salary_rating.csv', sep=";", decimal=",")
    salary_rating_df = salary_rating_df.set_index('Область')

    index_of_life_df = pd.read_excel('isg_2012-2023.xlsx', decimal='.', sheet_name=1)
    rf_regions_df = pd.read_excel('RF_region.xlsx', decimal='.')
    index_of_life_df = pd.merge(index_of_life_df, rf_regions_df, on=['Город'], how='left')
    index_of_life_df = index_of_life_df.groupby(by=['Регион'], as_index=False).mean()[['Регион', 2022, 2023]]
    index_of_life_df = index_of_life_df.rename(columns={'Регион': 'addrref', 2022: 'index_of_life_2022', 2023: 'index_of_life_2023'})
    index_of_life_df.loc[ len(index_of_life_df.index )] = ['Россия', 1.0, 1.0]

    df['addrref'] = df['addrref'].fillna('Россия')

    client_salary_month_df = df.join(salary_month_df, on='addrref')
    client_salary_month_df['feature_date'] = client_salary_month_df['feature_date'].astype('datetime64[ns]')
    client_salary_month_df['feature_date_month_year'] = client_salary_month_df['feature_date'].apply(lambda time: str(time.month) + '.' + str(time.year))
    client_salary_month_df['avg_salary_month'] = client_salary_month_df.apply(lambda x: x[x['feature_date_month_year']], axis=1)
    client_salary_month_df = client_salary_month_df[['client_id', 'avg_salary_month']]
    client_salary_month_df = client_salary_month_df.set_index('client_id')
    df = df.join(client_salary_month_df, on='client_id')

    df = df.join(vpm_df, on='addrref')

    df = df.join(salary_rating_df, on='addrref')

    df = pd.merge(df, index_of_life_df, on=['addrref'], how='left')
    return df

In [ ]:
def fill_nan(df, num_feats, cat_feats):
    for num_f in num_feats:
        df[num_f] = df[num_f].fillna(0.0).astype(float)
        df.loc[(df[num_f] == 'NaN') |
                (df[num_f] == 'None') |
                (df[num_f] == 'nan') |
                (df[num_f] == ''), num_f] = 0.0
    for cat_f in cat_feats:
        df[cat_f] = df[cat_f].fillna('no_data').astype(str)
        df.loc[(df[cat_f] == 'NaN') |
                (df[cat_f] == 'None') |
                (df[cat_f] == 'nan') |
                (df[cat_f] == ''), cat_f] = 'no_data'
    return df

In [ ]:
def fit_catboost_regressor(features_train, features_val, target_train, target_val, cat_features, params=None):
    default_params = {'learning_rate': 0.05, 'iterations': 500}
    if params is None:
        params = default_params

    pool_train = Pool(features_train, target_train, cat_features = cat_features)
    pool_val = None
    if features_val is not None:
        pool_val = Pool(features_val, target_val, cat_features = cat_features)

    model = CatBoostRegressor(**params, loss_function='RMSE', random_seed = 42, task_type="GPU")
    model.fit(pool_train, eval_set=pool_val, verbose=100, early_stopping_rounds=100)
    return model

In [ ]:
def fit_catboost_classifier(features_train, features_val, target_train, target_val, cat_features, params=None):
    default_params = {'learning_rate': 0.05, 'iterations': 500}
    if params is None:
        params = default_params

    pool_train = Pool(features_train, target_train, cat_features = cat_features)
    pool_val = None
    if features_val is not None:
        pool_val = Pool(features_val, target_val, cat_features = cat_features)

    model = CatBoostClassifier(**params, loss_function='MultiClass', random_seed = 42, task_type="GPU")
    model.fit(pool_train, eval_set=pool_val, verbose=100, early_stopping_rounds=100)
    return model

In [ ]:
def fit_xgboost_classifier(features_train, features_val, target_train, target_val, cat_features, params=None):
    default_params = {'learning_rate': 0.05, 'n_estimators': 100,
                      'booster': 'gbtree',
                      # 'tree_method':'gpu_hist',
                      'random_state': 42}
                      # "device": "cuda"}
    le = LabelEncoder()
    for feat in cat_features:
      features_train[feat] = le.fit_transform(features_train[feat])
    # features_train[cat_features] = features_train[cat_features].astype("category")
    if params is None:
        params = default_params
    eval_set = None
    if features_val is not None:
        # features_val[cat_features] = features_val[cat_features].astype("category")
        le = LabelEncoder()
        for feat in cat_features:
          features_val[feat] = le.fit_transform(features_val[feat])
        eval_set = [(features_train, target_train),(features_val, target_val)]

    model = XGBClassifier(**params, enable_categorical=True)
    model.fit(features_train, target_train, eval_set=eval_set, verbose=100)
    return model

# Чтение данных

In [ ]:
train_df = pd.read_csv('train.csv', sep=";", decimal=",", encoding="windows-1251")
test_df = pd.read_csv('test.csv', sep=";", decimal=",", encoding="windows-1251")
train_df.shape, test_df.shape

((205962, 235), (37183, 233))

In [ ]:
# salary_month_df, vpm_df = read_statistics_data()
# train_df = add_statistics_data_to_df(train_df, salary_month_df, vpm_df)
# test_df = add_statistics_data_to_df(test_df, salary_month_df, vpm_df)

In [ ]:
train_df = add_statistics_data_to_df(train_df)
test_df = add_statistics_data_to_df(test_df)

# Анализ данных. Отбор фичей

Разделяем на числовые и категориальные признаки

In [ ]:
# выбираем числовые признаки, которые ошибочны классифицировались категориальными
newtype_feats = ['bki_total_ip_max_limit',
 'hdb_bki_active_cc_cnt',
 'hdb_bki_active_ip_max_outstand',
 'hdb_bki_active_micro_max_outstand',
 'hdb_bki_active_pil_max_overdue',
 'hdb_bki_other_active_auto_month_payments_sum',
 'hdb_bki_total_cc_max_limit',
 'hdb_bki_total_ip_cnt',
 'hdb_bki_total_max_limit',
 'hdb_bki_total_max_overdue_sum',
 'hdb_bki_total_pil_max_limit']

In [ ]:
# изменяем тип данных у неправильно классифицированных признаков
train_df[newtype_feats] = train_df[newtype_feats].astype('float64')
train_df['feature_date'] = train_df['feature_date'].astype('datetime64[ns]')
train_df['feature_date_year'] = train_df['feature_date'].apply(lambda time: time.year)
train_df['feature_date_month'] = train_df['feature_date'].apply(lambda time: time.month)

In [ ]:
test_df[newtype_feats] = test_df[newtype_feats].astype('float64')
test_df['feature_date'] = test_df['feature_date'].astype('datetime64[ns]')
test_df['feature_date_year'] = test_df['feature_date'].apply(lambda time: time.year)
test_df['feature_date_month'] = test_df['feature_date'].apply(lambda time: time.month)

In [ ]:
all_feats = list(train_df.columns)
all_feats.remove('client_id')
all_feats.remove('target')
all_feats.remove('w')
all_feats.remove('feature_date')
len(all_feats)

241

In [ ]:
cat_feats = []
num_feats = []
for feat, type in zip(train_df[all_feats].dtypes.index, train_df[all_feats].dtypes):
  if type == object:
    cat_feats.append(feat)
  else:
    num_feats.append(feat)

len(num_feats), len(cat_feats)

(231, 10)

In [ ]:
# сколько всего записей, где известна должность, но не известна зп?
train_df.loc[
    (train_df['worksalary_rur_amt'].isna())
    & (~train_df['part_last_position_ccode'].isna())
].shape

(4184, 245)

In [ ]:
# df, содержащий медианные зарплаты для должностей по всем данным
median_worksalary = (
    pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
    .dropna(subset = 'worksalary_rur_amt')
    .groupby('part_last_position_ccode', as_index=False)
    ['worksalary_rur_amt'].median()
    .rename(columns={'worksalary_rur_amt': 'median_worksalary_rur_amt'})
)
median_worksalary.head()

,part_last_position_ccode,median_worksalary_rur_amt
0,АВТОМЕХАНИК,158892.5
1,АДМИНИСТРАТОР,180000.0
2,АНАЛИТИК,100000.0
3,Администратор,60000.0
4,Администратор магазина,70000.0


In [ ]:
test_df = test_df.merge(median_worksalary, on='part_last_position_ccode', how='left')
test_df['worksalary_rur_amt'] = test_df['worksalary_rur_amt'].fillna(test_df['median_worksalary_rur_amt'])
test_df.drop('median_worksalary_rur_amt', axis=1, inplace=True)

train_df = train_df.merge(median_worksalary, on='part_last_position_ccode', how='left')
train_df['worksalary_rur_amt'] = train_df['worksalary_rur_amt'].fillna(train_df['median_worksalary_rur_amt'])
train_df.drop('median_worksalary_rur_amt', axis=1, inplace=True)

train_df.loc[
    (train_df['worksalary_rur_amt'].isna())
    & (~train_df['part_last_position_ccode'].isna())
].shape

(1029, 245)

In [ ]:
# Фичи по датасету
import re
# машина
train_df['car_old'] = train_df['feature_date_year'] - train_df['productionyear']
train_df['car_country'] = train_df['brand']
train_df.loc[
    (train_df['car_country'].str.contains('LADA', flags=re.IGNORECASE))
    | (train_df['car_country'].str.contains('ЛАДА', flags=re.IGNORECASE))
    | (train_df['car_country'].isin(['МОСКВИЧ','КАМАЗ','ЗАЗ', 'ГАЗ'])),
    'car_country'
] = 'Россия'
train_df.loc[
    ~train_df['car_country'].isin(['miss_value', 'Россия']),
    'car_country'
] = 'Иномарка'

test_df['car_old'] = test_df['feature_date_year'] - test_df['productionyear']
test_df['car_country'] = test_df['brand']
test_df.loc[
    (test_df['car_country'].str.contains('LADA', flags=re.IGNORECASE))
    | (test_df['car_country'].str.contains('ЛАДА', flags=re.IGNORECASE))
    | (test_df['car_country'].isin(['МОСКВИЧ','КАМАЗ','ЗАЗ', 'ГАЗ'])),
    'car_country'
] = 'Россия'
test_df.loc[
    ~test_df['car_country'].isin(['miss_value', 'Россия']),
    'car_country'
] = 'Иномарка'

# средняя, медианная, минимальная, максимальная зарплата по должности
train_df = (
    train_df
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('part_last_position_ccode', as_index=False)
        ['worksalary_rur_amt'].median()
        .rename(columns={'worksalary_rur_amt': 'position_median_worksalary_rur_amt'}),
        on='part_last_position_ccode',
        how='left'
    )
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('part_last_position_ccode', as_index=False)
        ['worksalary_rur_amt'].mean()
        .rename(columns={'worksalary_rur_amt': 'position_mean_worksalary_rur_amt'}),
        on='part_last_position_ccode',
        how='left'
    )
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('part_last_position_ccode', as_index=False)
        ['worksalary_rur_amt'].max()
        .rename(columns={'worksalary_rur_amt': 'position_max_worksalary_rur_amt'}),
        on='part_last_position_ccode',
        how='left'
    )
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('part_last_position_ccode', as_index=False)
        ['worksalary_rur_amt'].min()
        .rename(columns={'worksalary_rur_amt': 'position_min_worksalary_rur_amt'}),
        on='part_last_position_ccode',
        how='left'
    )
)

test_df = (
    test_df
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('part_last_position_ccode', as_index=False)
        ['worksalary_rur_amt'].median()
        .rename(columns={'worksalary_rur_amt': 'position_median_worksalary_rur_amt'}),
        on='part_last_position_ccode',
        how='left'
    )
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('part_last_position_ccode', as_index=False)
        ['worksalary_rur_amt'].mean()
        .rename(columns={'worksalary_rur_amt': 'position_mean_worksalary_rur_amt'}),
        on='part_last_position_ccode',
        how='left'
    )
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('part_last_position_ccode', as_index=False)
        ['worksalary_rur_amt'].max()
        .rename(columns={'worksalary_rur_amt': 'position_max_worksalary_rur_amt'}),
        on='part_last_position_ccode',
        how='left'
    )
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('part_last_position_ccode', as_index=False)
        ['worksalary_rur_amt'].min()
        .rename(columns={'worksalary_rur_amt': 'position_min_worksalary_rur_amt'}),
        on='part_last_position_ccode',
        how='left'
    )
)

# средняя, медианная, минимальная, максимальная зарплата по месту жительства
train_df = (
    train_df
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('addrref', as_index=False)
        ['worksalary_rur_amt'].median()
        .rename(columns={'worksalary_rur_amt': 'addrref_median_worksalary_rur_amt'}),
        on='addrref',
        how='left'
    )
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('addrref', as_index=False)
        ['worksalary_rur_amt'].mean()
        .rename(columns={'worksalary_rur_amt': 'addrref_mean_worksalary_rur_amt'}),
        on='addrref',
        how='left'
    )
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('addrref', as_index=False)
        ['worksalary_rur_amt'].max()
        .rename(columns={'worksalary_rur_amt': 'addrref_max_worksalary_rur_amt'}),
        on='addrref',
        how='left'
    )
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('addrref', as_index=False)
        ['worksalary_rur_amt'].min()
        .rename(columns={'worksalary_rur_amt': 'addrref_min_worksalary_rur_amt'}),
        on='addrref',
        how='left'
    )
)

test_df = (
    test_df
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('addrref', as_index=False)
        ['worksalary_rur_amt'].median()
        .rename(columns={'worksalary_rur_amt': 'addrref_median_worksalary_rur_amt'}),
        on='addrref',
        how='left'
    )
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('addrref', as_index=False)
        ['worksalary_rur_amt'].mean()
        .rename(columns={'worksalary_rur_amt': 'addrref_mean_worksalary_rur_amt'}),
        on='addrref',
        how='left'
    )
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('addrref', as_index=False)
        ['worksalary_rur_amt'].max()
        .rename(columns={'worksalary_rur_amt': 'addrref_max_worksalary_rur_amt'}),
        on='addrref',
        how='left'
    )
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('addrref', as_index=False)
        ['worksalary_rur_amt'].min()
        .rename(columns={'worksalary_rur_amt': 'addrref_min_worksalary_rur_amt'}),
        on='addrref',
        how='left'
    )
)

# средняя, медианная, минимальная, максимальная зарплата по сегменту
train_df = (
    train_df
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('segment', as_index=False)
        ['worksalary_rur_amt'].median()
        .rename(columns={'worksalary_rur_amt': 'segment_median_worksalary_rur_amt'}),
        on='segment',
        how='left'
    )
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('segment', as_index=False)
        ['worksalary_rur_amt'].mean()
        .rename(columns={'worksalary_rur_amt': 'segment_mean_worksalary_rur_amt'}),
        on='segment',
        how='left'
    )
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('segment', as_index=False)
        ['worksalary_rur_amt'].max()
        .rename(columns={'worksalary_rur_amt': 'segment_max_worksalary_rur_amt'}),
        on='segment',
        how='left'
    )
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('segment', as_index=False)
        ['worksalary_rur_amt'].min()
        .rename(columns={'worksalary_rur_amt': 'segment_min_worksalary_rur_amt'}),
        on='segment',
        how='left'
    )
)

test_df = (
    test_df
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('segment', as_index=False)
        ['worksalary_rur_amt'].median()
        .rename(columns={'worksalary_rur_amt': 'segment_median_worksalary_rur_amt'}),
        on='segment',
        how='left'
    )
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('segment', as_index=False)
        ['worksalary_rur_amt'].mean()
        .rename(columns={'worksalary_rur_amt': 'segment_mean_worksalary_rur_amt'}),
        on='segment',
        how='left'
    )
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('segment', as_index=False)
        ['worksalary_rur_amt'].max()
        .rename(columns={'worksalary_rur_amt': 'segment_max_worksalary_rur_amt'}),
        on='segment',
        how='left'
    )
    .merge(
        pd.concat([train_df[all_feats], test_df[all_feats]], axis=0)
        .groupby('segment', as_index=False)
        ['worksalary_rur_amt'].min()
        .rename(columns={'worksalary_rur_amt': 'segment_min_worksalary_rur_amt'}),
        on='segment',
        how='left'
    )
)

In [ ]:
all_feats = list(train_df.columns)
all_feats.remove('client_id')
all_feats.remove('target')
all_feats.remove('w')
all_feats.remove('feature_date')
len(all_feats)

cat_feats = []
num_feats = []
for feat, type in zip(train_df[all_feats].dtypes.index, train_df[all_feats].dtypes):
  if type == object:
    cat_feats.append(feat)
  else:
    num_feats.append(feat)

len(num_feats), len(cat_feats)

(244, 11)

In [ ]:
all_feats

['accum_rur_amt_cm_avg_div_v2',
 'addrref',
 'amount_by_category_30d__summarur_amt__sum__cashflowcategory_name__bilety_na_kontserty_i_v_teatry',
 'amount_by_category_30d__summarur_amt__sum__cashflowcategory_name__brokerskie_uslugi',
 'amount_by_category_30d__summarur_amt__sum__cashflowcategory_name__hosting',
 'amount_by_category_30d__summarur_amt__sum__cashflowcategory_name__spa_sauny_bani',
 'amount_by_category_30d__summarur_amt__sum__cashflowcategory_name__tovary_dlja_detej',
 'amount_by_category_30d__summarur_amt__sum__cashflowcategory_name__turisticheskie_agenstva',
 'amount_by_category_90d__summarur_amt__sum__cashflowcategory_name__marketplejsy',
 'amount_by_category_90d__summarur_amt__sum__cashflowcategory_name__nalogi',
 'amount_by_category_90d__summarur_amt__sum__cashflowcategory_name__ohota_i_rybalka',
 'amount_by_category_90d__summarur_amt__sum__cashflowcategory_name__prochie_bilety',
 'atravel',
 'avg_3m_hotels',
 'avg_6m_building_services',
 'avg_6m_money_transactions',
 '

## Отбираем наиболее важные признаки

In [ ]:
# по пропущенным значениям

def feats_filter_nan(df: pd.DataFrame,
                     num_feat: list(),
                     cat_feat: list(),
                     nan_threshold: float):

    """Функция отбора фичей по доле NaN-значений.
    Фичи, которые имеют долю NaN-значений не ниже заданного nan_threshold, подлежат удалению.

    Parameters
    ----------
    df : pd.DataFrame
        датафрейм с фичами
    num_feat : list
        список числовых фичей
    cat_feat : list
        список категориальных фичей (в нашем случае для них NaN-значением является 'no_data')
    nan_threshold : float
        порог удаления фичи

    Returns
    -------
    list
        список фичей, которые останутся в датафрейме

    """
    selected_num_feat = (df[num_feat].isna().sum()/df.shape[0]).loc[lambda x: x < nan_threshold].index.values
    selected_cat_feat = (df[cat_feat].isin(['no_data']).sum()/df.shape[0]).loc[lambda x: x < nan_threshold].index.values
    res = list(selected_num_feat) + list(selected_cat_feat)

    return res

selected_features = feats_filter_nan(train_df, num_feats, cat_feats, 0.9)
selected_features_num = [feat for feat in num_feats if feat in selected_features]
selected_features_cat = [feat for feat in cat_feats if feat in selected_features]
len(selected_features)

139

In [ ]:
# Заполняем пропуски
train_df = fill_nan(train_df, num_feats, cat_feats)
test_df = fill_nan(test_df, num_feats, cat_feats)

In [ ]:
# по доле самого частого значения

def feats_filter_nunique(df: pd.DataFrame,
                         features: list,
                         unique_threshold: float):

    """Функция отбора фичей по доле самого частого значения.
    Фичи, которые имеют 1 уникальное значение (помимо NaN), либо для которых
      доля самого частого значения (включая NaN) не ниже unique_threshold, подлежат удалению.

    Parameters
    ----------
    df : pd.DataFrame
        датафрейм с фичами
    features : list
        список фичей
    unique_threshold : float
        порог удаления фичи

    Returns
    -------
    list
        список фичей, которые останутся в датафрейме

    """
    res = []
    for feature in features:
      series = df[feature].value_counts(dropna=False, normalize=True)
      if len(series.index.dropna()) == 1:
        continue
      if series.max() < unique_threshold:
        res.append(feature)

    return res

selected_features = feats_filter_nunique(train_df, selected_features, 0.9)
selected_features_num = [feat for feat in num_feats if feat in selected_features]
selected_features_cat = [feat for feat in cat_feats if feat in selected_features]
len(selected_features)

100

In [ ]:
selected_features

['accum_rur_amt_cm_avg_div_v2',
 'avg_6m_building_services',
 'avg_6m_money_transactions',
 'avg_6m_personal_services',
 'avg_6m_transportation',
 'avg_by_category__amount__sum__cashflowcategory_name__vydacha_nalichnyh_v_bankomate',
 'avg_by_category__amount__sum__cashflowcategory_name__zdorove',
 'avg_cnt_daily_transactions_90d',
 'avg_debet_turn_rur',
 'by_category__amount__sum__eoperation_type_name__vneshnij_perevod_rur',
 'calls_weight_v2',
 'card_dda_rur_amt_cm_avg_div_v2',
 'cc_other_rate_max_2avg_prop',
 'channel_chat_s_operatorom_voc_features_12m_cnt_mark',
 'channel_chat_s_operatorom_voc_features_full_voc_without_marks_portion',
 'channel_chatbot_voc_features_full_voc_without_marks_portion',
 'channel_ivr_voc_features_36m_voc_not_null_flag',
 'channel_otdelenija_voc_features_full_voc_not_null_flag',
 'channel_telemarketing_voc_features_3m_voc_not_null_flag',
 'cntOnnRinCallAvg6m',
 'commission_outcome_rur_amt',
 'count_device_huawei_0_365_days',
 'count_device_samsung_0_60_day

In [ ]:
# по корреляции

def feats_filter_corr(df: pd.DataFrame,
                      features: list,
                      corr_threshold: float):

    """Функция отбора фичей по корреляции между фичами.
    Фичи, которые коррелируют с показателем не менее corr_threshold, подлежат
      удалению (из пары коррелирующих фичей удаляется менее заполненная фича).
      Используется метод df[features].corr() с корреляцией Пирсона.

    Parameters
    ----------
    df : pd.DataFrame
        датафрейм с фичами
    features : list
        список фичей
    corr_threshold : float
        порог удаления фичи

    Returns
    -------
    list
        список фичей, которые останутся в датафрейме

    """
    cat_features = df[features].select_dtypes(include=['object']).columns
    df_tmp = df[features]
    df_tmp[cat_features] = df[cat_features].apply(lambda x: pd.factorize(x)[0])
    features_corr = df_tmp.corr()
    res = features.copy()

    def delete_in_selected_features(res_list: list, worst_feature: str):
      if worst_feature in res_list:
        res_list.remove(worst_feature)

    for i in range(len(features)-1):
      for j in range(i+1, len(features)):
        if abs(features_corr.iloc[i][j]) >= corr_threshold:
          if features[i] in cat_features and features[j] in cat_features:
            if df[features[i]].isin(['no_data']).sum() <= df[features[j]].isin(['no_data']).sum():
              delete_in_selected_features(res, features[j])
            else:
              delete_in_selected_features(res, features[i])

          elif features[i] in cat_features and features[j] not in cat_features:
            if df[features[i]].isin(['no_data']).sum() < df[features[j]].isna().sum():
              delete_in_selected_features(res, features[j])
            else:
              delete_in_selected_features(res, features[i])

          elif features[i] not in cat_features and features[j] in cat_features:
            if df[features[i]].isna().sum() < df[features[j]].isin(['no_data']).sum():
              delete_in_selected_features(res, features[j])
            else:
              delete_in_selected_features(res, features[i])

          else:
            if df[features[i]].isna().sum() < df[features[j]].isna().sum():
              delete_in_selected_features(res, features[j])
            else:
              delete_in_selected_features(res, features[i])
    return res

selected_features = feats_filter_corr(train_df, selected_features, 0.95)
selected_features_num = [feat for feat in num_feats if feat in selected_features]
selected_features_cat = [feat for feat in cat_feats if feat in selected_features]
len(selected_features)

92

In [ ]:
selected_features

['accum_rur_amt_cm_avg_div_v2',
 'avg_6m_building_services',
 'avg_6m_money_transactions',
 'avg_6m_personal_services',
 'avg_6m_transportation',
 'avg_by_category__amount__sum__cashflowcategory_name__vydacha_nalichnyh_v_bankomate',
 'avg_by_category__amount__sum__cashflowcategory_name__zdorove',
 'avg_cnt_daily_transactions_90d',
 'avg_debet_turn_rur',
 'by_category__amount__sum__eoperation_type_name__vneshnij_perevod_rur',
 'calls_weight_v2',
 'card_dda_rur_amt_cm_avg_div_v2',
 'cc_other_rate_max_2avg_prop',
 'channel_chat_s_operatorom_voc_features_12m_cnt_mark',
 'channel_chat_s_operatorom_voc_features_full_voc_without_marks_portion',
 'channel_chatbot_voc_features_full_voc_without_marks_portion',
 'channel_ivr_voc_features_36m_voc_not_null_flag',
 'channel_otdelenija_voc_features_full_voc_not_null_flag',
 'channel_telemarketing_voc_features_3m_voc_not_null_flag',
 'cntOnnRinCallAvg6m',
 'commission_outcome_rur_amt',
 'count_device_huawei_0_365_days',
 'count_device_samsung_0_60_day

In [ ]:
filtered_features = selected_features.copy()
filtered_features_num = selected_features_num.copy()
filtered_features_cat = selected_features_cat.copy()

# Выбор признаков и обучение классификатора

По результатам предварительного анализа решено оставить 3 бина

In [ ]:
# вариант 1
train_df['target_bins'] = pd.qcut(train_df['target'], q=3)
train_df['target_bins'] = train_df['target_bins'].astype('string')

In [ ]:
le = LabelEncoder()
train_df['target_bins'] = le.fit_transform(train_df['target_bins'])

In [ ]:
# отсечение по рандомной фиче

def feats_filter_random_feat(df: pd.DataFrame,
                             features: list,
                             cat_features: list):

    """Функция отбора фичей при помощи отсечения менее важных фичей чем рандомная фича.
    Фичи, которые имеют в простой модели важность ниже рандомной фичи, подлежат удалению.

    Parameters
    ----------
    df : pd.DataFrame
        датафрейм с фичами
    features : list
        список фичей
    cat_features : list
        список категориальных фичей

    Returns
    -------
    list
        список фичей, которые останутся в датафрейме

    """

    df['random'] = np.random.uniform(0, 1, df.shape[0])
    simple_model = fit_catboost_classifier(df[features + ['random']], None, df['target_bins'], None, cat_features)
    feature_importance = pd.Series(simple_model.get_feature_importance(), df[features + ['random']].columns)
    selected_features = list(feature_importance[feature_importance > feature_importance['random']].index)
    df = df.drop('random', axis=1)
    return selected_features, feature_importance

train_df = train_df.sample(frac=1)
classifier_features, feature_importance = feats_filter_random_feat(train_df, selected_features, selected_features_cat)
classifier_features_num = [feat for feat in num_feats if feat in classifier_features]
classifier_features_cat = [feat for feat in cat_feats if feat in classifier_features]
len(classifier_features)

0:	learn: 1.0745351	total: 29.4ms	remaining: 14.7s
100:	learn: 0.7200470	total: 1.52s	remaining: 5.99s
200:	learn: 0.6865376	total: 3.96s	remaining: 5.89s
300:	learn: 0.6717380	total: 6.73s	remaining: 4.45s
400:	learn: 0.6625552	total: 8s	remaining: 1.98s
499:	learn: 0.6558130	total: 9.28s	remaining: 0us


69

In [ ]:
classifier_features

['accum_rur_amt_cm_avg_div_v2',
 'avg_6m_building_services',
 'avg_6m_money_transactions',
 'avg_6m_personal_services',
 'avg_6m_transportation',
 'avg_by_category__amount__sum__cashflowcategory_name__vydacha_nalichnyh_v_bankomate',
 'avg_by_category__amount__sum__cashflowcategory_name__zdorove',
 'avg_cnt_daily_transactions_90d',
 'avg_debet_turn_rur',
 'by_category__amount__sum__eoperation_type_name__vneshnij_perevod_rur',
 'cc_other_rate_max_2avg_prop',
 'channel_chatbot_voc_features_full_voc_without_marks_portion',
 'channel_ivr_voc_features_36m_voc_not_null_flag',
 'channel_otdelenija_voc_features_full_voc_not_null_flag',
 'channel_telemarketing_voc_features_3m_voc_not_null_flag',
 'cntOnnRinCallAvg6m',
 'commission_outcome_rur_amt',
 'count_device_huawei_0_365_days',
 'count_device_samsung_0_60_days',
 'curr_rur_amt_cm_avg',
 'flg_clip_cc_presence_in_previous_30_days_with_150_offset',
 'hdb_bki_active_ip_max_outstand',
 'hdb_bki_active_pil_max_overdue',
 'hdb_bki_total_cc_max_lim

In [ ]:
classifier_features = ['accum_rur_amt_cm_avg_div_v2',
 'avg_6m_building_services',
 'avg_6m_money_transactions',
 'avg_6m_personal_services',
 'avg_6m_transportation',
 'avg_by_category__amount__sum__cashflowcategory_name__vydacha_nalichnyh_v_bankomate',
 'avg_by_category__amount__sum__cashflowcategory_name__zdorove',
 'avg_cnt_daily_transactions_90d',
 'avg_debet_turn_rur',
 'cc_other_rate_max_2avg_prop',
 'channel_chat_s_operatorom_voc_features_12m_cnt_mark',
 'channel_chatbot_voc_features_full_voc_without_marks_portion',
 'channel_otdelenija_voc_features_full_voc_not_null_flag',
 'cntOnnRinCallAvg6m',
 'commission_outcome_rur_amt',
 'curr_rur_amt_cm_avg',
 'hdb_bki_active_cc_cnt',
 'hdb_bki_active_ip_max_outstand',
 'hdb_bki_active_pil_max_overdue',
 'hdb_bki_total_cc_max_limit',
 'hdb_bki_total_ip_cnt',
 'hdb_bki_total_max_limit',
 'hdb_bki_total_max_overdue_sum',
 'hdb_bki_total_pil_max_limit',
 'hdb_outstand_sum',
 'incomeValue',
 'max_cc_largest_max_limit_actoff_30d',
 'max_pil_largest_max_limit_actoff_90d',
 'min_cc_max_el_actoff_90d',
 'min_cc_offer_lifetime_days_actoff_90d',
 'min_max_limit',
 'min_pil_max_score_actoff_180d',
 'min_pil_max_ul_actoff_90d',
 'mob_cover_days',
 'percent_outcome_rur_amt',
 'product_zarplatnaja_karta_voc_features_36m_voc_without_marks_portion',
 'prof_cc_prof',
 'profit_income_out_rur_amt_9m',
 'puupg_offer_exists_pass_180d',
 'smsInWavg6m',
 'summarur_1m_no_cat',
 'top_channel_weight_v2',
 'total_rur_amt_cm_avg_div_v2',
 'total_rur_amt_cm_avg_period_days_ago_v2',
 'transaction_category_general_store_inc_amt_15d',
 'transaction_category_general_store_inc_cnt_2m',
 'transaction_category_house_repair_inc_cnt_2m',
 'transaction_category_money_send_sum_cnt_m2',
 'transaction_category_other_retail_purchase_percent_amt_2m',
 'transaction_category_supermarket_sum_amt_m3_4',
 'transaction_category_supermarket_sum_cnt_d15',
 'turn_cc_cr_max_v2',
 'turn_cc_db_sum_v2',
 'turn_other_cr_avg_v2',
 'turn_other_db_max_v2',
 'uniV5',
 'voc_features_12m_voc_without_marks_portion',
 'voc_features_36m_mark_eq_1_flag',
 'voc_features_36m_max_mark',
 'worksalary_rur_amt',
 'avg_salary_month',
 'ratio_less_200',
 'index_of_life_2023',
 'feature_date_year',
 'feature_date_month',
 'addrref_median_worksalary_rur_amt',
 'addrref_mean_worksalary_rur_amt',
 'addrref_max_worksalary_rur_amt',
 'addrref_min_worksalary_rur_amt',
 'segment_mean_worksalary_rur_amt',
 'segment_max_worksalary_rur_amt',
 'segment_min_worksalary_rur_amt',
 'addrref',
 'main_last_position_ccode',
 'main_pre_last_position_ccode',
 'oldest_campaignsegment_ccode_for_nss',
 'oldest_campaignsegment_ccode_for_pil',
 'segment']
classifier_features_num = [feat for feat in num_feats if feat in classifier_features]
classifier_features_cat = [feat for feat in cat_feats if feat in classifier_features]

## Обучение классификатора

In [ ]:
train_df = train_df.sample(frac=1)
# разделим данные на обучающую, тестовую и валидационную выборки
x_train, x_val, y_train, y_val = train_test_split(train_df, train_df['target_bins'], test_size=0.15, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x_train, x_train['target_bins'], test_size=0.15, random_state=42)

In [ ]:
# import optuna

# def objective(trial):

#     # 1. назначаем параметры и значения для перебора
#     params = {
#         "iterations": 2000,
#         "learning_rate": trial.suggest_float("learning_rate", 1e-4, 0.1),
#         "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.1, 10.0),
#         "depth": trial.suggest_int("depth", 4, 11),
#         "random_strength": trial.suggest_float("random_strength", 0.1, 5.0),
#     }

#     # 2. обучаем модель с выбранными параметрами
#     model_cv = fit_catboost_classifier(x_train[classifier_features], x_val[classifier_features], y_train, y_val, classifier_features_cat, params)

#     # 3. делаем предсказание и вовращаем значение метрики
#     preds = model_cv.predict(x_test[classifier_features])
#     score = accuracy_score(y_test, preds)
#     return score

In [ ]:
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=10)

[I 2024-02-18 23:14:16,440] A new study created in memory with name: no-name-c6613f8d-6e89-4f45-8292-22f0d01f0277


0:	learn: 1.0661707	test: 1.0661988	best: 1.0661988 (0)	total: 22.5ms	remaining: 44.9s
100:	learn: 0.6877812	test: 0.6995548	best: 0.6995548 (100)	total: 1.89s	remaining: 35.6s
200:	learn: 0.6546085	test: 0.6768667	best: 0.6768667 (200)	total: 3.73s	remaining: 33.4s
300:	learn: 0.6356183	test: 0.6686571	best: 0.6686571 (300)	total: 5.58s	remaining: 31.5s
400:	learn: 0.6213026	test: 0.6639452	best: 0.6639452 (400)	total: 7.41s	remaining: 29.6s
500:	learn: 0.6090407	test: 0.6614345	best: 0.6614231 (499)	total: 9.25s	remaining: 27.7s
600:	learn: 0.5980762	test: 0.6594525	best: 0.6594525 (600)	total: 11.1s	remaining: 25.8s
700:	learn: 0.5877508	test: 0.6581859	best: 0.6581859 (700)	total: 12.9s	remaining: 24s
800:	learn: 0.5783044	test: 0.6571146	best: 0.6571146 (800)	total: 14.8s	remaining: 22.1s
900:	learn: 0.5685977	test: 0.6562673	best: 0.6562628 (899)	total: 16.6s	remaining: 20.3s
1000:	learn: 0.5594099	test: 0.6556717	best: 0.6556717 (1000)	total: 18.5s	remaining: 18.4s
1100:	learn: 

[I 2024-02-18 23:14:51,275] Trial 0 finished with value: 0.7158524047065992 and parameters: {'learning_rate': 0.060868423216076085, 'l2_leaf_reg': 2.0864465214142847, 'depth': 8, 'random_strength': 0.5706729126704134}. Best is trial 0 with value: 0.7158524047065992.


0:	learn: 1.0977454	test: 1.0977463	best: 1.0977463 (0)	total: 9.25ms	remaining: 18.5s
100:	learn: 1.0286326	test: 1.0287667	best: 1.0287667 (100)	total: 901ms	remaining: 16.9s
200:	learn: 0.9821031	test: 0.9825101	best: 0.9825101 (200)	total: 1.78s	remaining: 16s
300:	learn: 0.9476831	test: 0.9483090	best: 0.9483090 (300)	total: 2.65s	remaining: 15s
400:	learn: 0.9208816	test: 0.9216735	best: 0.9216735 (400)	total: 3.52s	remaining: 14s
500:	learn: 0.8992175	test: 0.9000392	best: 0.9000392 (500)	total: 4.38s	remaining: 13.1s
600:	learn: 0.8811537	test: 0.8819916	best: 0.8819916 (600)	total: 5.25s	remaining: 12.2s
700:	learn: 0.8654469	test: 0.8664214	best: 0.8664214 (700)	total: 6.1s	remaining: 11.3s
800:	learn: 0.8519915	test: 0.8530573	best: 0.8530573 (800)	total: 6.96s	remaining: 10.4s
900:	learn: 0.8404649	test: 0.8416560	best: 0.8416560 (900)	total: 7.83s	remaining: 9.54s
1000:	learn: 0.8305697	test: 0.8318778	best: 0.8318778 (1000)	total: 8.68s	remaining: 8.66s
1100:	learn: 0.821

[I 2024-02-18 23:15:09,936] Trial 1 finished with value: 0.6707665359278017 and parameters: {'learning_rate': 0.002038738665787424, 'l2_leaf_reg': 8.85294504020333, 'depth': 4, 'random_strength': 3.080541170163515}. Best is trial 0 with value: 0.7158524047065992.


...

[I 2024-02-18 23:18:34,325] Trial 7 finished with value: 0.7021819428049199 and parameters: {'learning_rate': 0.00606373691505543, 'l2_leaf_reg': 4.417074580192249, 'depth': 7, 'random_strength': 0.6734195705757522}. Best is trial 0 with value: 0.7158524047065992.


0:	learn: 1.0851043	test: 1.0850934	best: 1.0850934 (0)	total: 13.1ms	remaining: 26.1s
100:	learn: 0.7649127	test: 0.7686121	best: 0.7686121 (100)	total: 1.22s	remaining: 23s
200:	learn: 0.7149932	test: 0.7207227	best: 0.7207227 (200)	total: 2.41s	remaining: 21.6s
300:	learn: 0.6943055	test: 0.7017089	best: 0.7017089 (300)	total: 3.59s	remaining: 20.3s
400:	learn: 0.6818781	test: 0.6910052	best: 0.6910052 (400)	total: 4.78s	remaining: 19s
500:	learn: 0.6736432	test: 0.6845205	best: 0.6845205 (500)	total: 5.95s	remaining: 17.8s
600:	learn: 0.6670576	test: 0.6795747	best: 0.6795747 (600)	total: 7.12s	remaining: 16.6s
700:	learn: 0.6617795	test: 0.6760286	best: 0.6760286 (700)	total: 8.3s	remaining: 15.4s
800:	learn: 0.6572158	test: 0.6731612	best: 0.6731612 (800)	total: 9.48s	remaining: 14.2s
900:	learn: 0.6536196	test: 0.6710422	best: 0.6710422 (900)	total: 10.7s	remaining: 13s
1000:	learn: 0.6503746	test: 0.6692019	best: 0.6692019 (1000)	total: 11.8s	remaining: 11.8s
1100:	learn: 0.647

[I 2024-02-18 23:18:59,169] Trial 8 finished with value: 0.7118160009139027 and parameters: {'learning_rate': 0.028057111962909397, 'l2_leaf_reg': 8.633721182893757, 'depth': 6, 'random_strength': 2.761495918999857}. Best is trial 0 with value: 0.7158524047065992.


0:	learn: 1.0781905	test: 1.0782310	best: 1.0782310 (0)	total: 9.49ms	remaining: 19s
100:	learn: 0.7528549	test: 0.7557773	best: 0.7557773 (100)	total: 872ms	remaining: 16.4s
200:	learn: 0.7113935	test: 0.7161078	best: 0.7161078 (200)	total: 1.73s	remaining: 15.5s
300:	learn: 0.6943616	test: 0.7003509	best: 0.7003509 (300)	total: 2.58s	remaining: 14.6s
400:	learn: 0.6836071	test: 0.6907066	best: 0.6907066 (400)	total: 3.44s	remaining: 13.7s
500:	learn: 0.6763304	test: 0.6847398	best: 0.6847398 (500)	total: 4.3s	remaining: 12.9s
600:	learn: 0.6707365	test: 0.6805445	best: 0.6805445 (600)	total: 5.15s	remaining: 12s
700:	learn: 0.6662723	test: 0.6774311	best: 0.6774311 (700)	total: 6s	remaining: 11.1s
800:	learn: 0.6626052	test: 0.6751036	best: 0.6751036 (800)	total: 6.86s	remaining: 10.3s
900:	learn: 0.6592296	test: 0.6729111	best: 0.6729111 (900)	total: 7.71s	remaining: 9.41s
1000:	learn: 0.6562116	test: 0.6710855	best: 0.6710841 (999)	total: 8.57s	remaining: 8.55s
1100:	learn: 0.65358

[I 2024-02-18 23:19:17,691] Trial 9 finished with value: 0.7109020981683866 and parameters: {'learning_rate': 0.04894241540418253, 'l2_leaf_reg': 0.6771495817517926, 'depth': 4, 'random_strength': 4.782621579660648}. Best is trial 0 with value: 0.7158524047065992.


In [ ]:
params = {"iterations": 8000, 'learning_rate': 0.04, 'l2_leaf_reg': 2.911507368, 'depth': 7, 'random_strength': 0.49193290791022337}
# params = {"iterations": 8000, 'learning_rate': 0.03, 'l2_leaf_reg': 2.911507368, 'depth': 7, 'random_strength': 0.49193290791022337} # my
# params = {"iterations": 4000, 'learning_rate': 0.060868423216076085, 'l2_leaf_reg': 2.0864465214142847, 'depth': 8, 'random_strength': 0.5706729126704134}

classifier_model = fit_catboost_classifier(x_train[classifier_features], x_val[classifier_features], y_train, y_val, classifier_features_cat, params)
classifier_model.save_model("classifier_model.model")

0:	learn: 1.0781674	test: 1.0785951	best: 1.0785951 (0)	total: 28.1ms	remaining: 3m 44s
100:	learn: 0.7245670	test: 0.7294411	best: 0.7294411 (100)	total: 1.71s	remaining: 2m 13s
200:	learn: 0.6861689	test: 0.6941798	best: 0.6941798 (200)	total: 3.24s	remaining: 2m 5s
300:	learn: 0.6697635	test: 0.6815013	best: 0.6815013 (300)	total: 4.74s	remaining: 2m 1s
400:	learn: 0.6580479	test: 0.6743037	best: 0.6743037 (400)	total: 6.24s	remaining: 1m 58s
500:	learn: 0.6495585	test: 0.6695517	best: 0.6695517 (500)	total: 7.73s	remaining: 1m 55s
600:	learn: 0.6424988	test: 0.6664856	best: 0.6664856 (600)	total: 11.9s	remaining: 2m 26s
700:	learn: 0.6363812	test: 0.6640604	best: 0.6640604 (700)	total: 13.4s	remaining: 2m 20s
800:	learn: 0.6308175	test: 0.6623810	best: 0.6623810 (800)	total: 14.9s	remaining: 2m 14s
900:	learn: 0.6256447	test: 0.6608588	best: 0.6608588 (900)	total: 16.5s	remaining: 2m 9s
1000:	learn: 0.6207169	test: 0.6597864	best: 0.6597864 (1000)	total: 17.9s	remaining: 2m 5s
1100

In [ ]:
preds = classifier_model.predict(x_test[classifier_features])
score = accuracy_score(y_test, preds)
score

0.7148623433989566

In [ ]:
# classifier_model = CatBoostClassifier()
# classifier_model.load_model('classifier_model.model')

## Использование обученного классификатора

In [ ]:
# classifier_model = CatBoostClassifier()
# classifier_model.load_model('best_models/classifier_model.model')
# classifier_features = classifier_model.feature_names_
# classifier_features_cat = [feat for feat in cat_feats if feat in classifier_features]

In [ ]:
# preds = classifier_model.predict(x_test[classifier_features_best])
# score = accuracy_score(y_test, preds)
# score

##Дополнение данных вероятностями класса


In [ ]:
train_df['cl0_prob'] = classifier_model.predict_proba(train_df[classifier_model.feature_names_])[:,0]
train_df['cl1_prob'] = classifier_model.predict_proba(train_df[classifier_model.feature_names_])[:,1]
train_df['cl2_prob'] = classifier_model.predict_proba(train_df[classifier_model.feature_names_])[:,2]
train_df['target_bins'] = train_df['target_bins'].astype('string')

test_df['target_bins'] = classifier_model.predict(test_df[classifier_model.feature_names_])[:,-1]
test_df['target_bins'] = test_df['target_bins'].astype('string')
test_df['cl0_prob'] = classifier_model.predict_proba(test_df[classifier_model.feature_names_])[:,0]
test_df['cl1_prob'] = classifier_model.predict_proba(test_df[classifier_model.feature_names_])[:,1]
test_df['cl2_prob'] = classifier_model.predict_proba(test_df[classifier_model.feature_names_])[:,2]

# Выбор признаков и обучение регрессора

In [ ]:
selected_features = filtered_features.copy()
selected_features_num = filtered_features_num.copy()
selected_features_cat = filtered_features_cat.copy()

if 'target_bins' not in selected_features:
    selected_features.append('target_bins')
    selected_features_cat.append('target_bins')
    cat_feats.append('target_bins')

for prob in ['cl0_prob', 'cl1_prob', 'cl2_prob']:
  if prob not in selected_features:
    selected_features.append(prob)
    selected_features_num.append(prob)
    num_feats.append(prob)


len(selected_features), len(selected_features_cat)

(86, 7)

In [ ]:
# отсечение по рандомной фиче

def feats_filter_random_feat(df: pd.DataFrame,
                             features: list,
                             cat_features: list):

    """Функция отбора фичей при помощи отсечения менее важных фичей чем рандомная фича.
    Фичи, которые имеют в простой модели важность ниже рандомной фичи, подлежат удалению.

    Parameters
    ----------
    df : pd.DataFrame
        датафрейм с фичами
    features : list
        список фичей
    cat_features : list
        список категориальных фичей

    Returns
    -------
    list
        список фичей, которые останутся в датафрейме

    """

    df['random'] = np.random.uniform(0, 1, df.shape[0])
    simple_model = fit_catboost_regressor(df[features + ['random']], None, df['target'], None, cat_features)
    feature_importance = pd.Series(simple_model.get_feature_importance(), df[features + ['random']].columns)
    selected_features = list(feature_importance[feature_importance > feature_importance['random']].index)
    df = df.drop('random', axis=1)
    return selected_features, feature_importance

train_df = train_df.sample(frac=1)
regressor_features, feature_importance = feats_filter_random_feat(train_df, selected_features, selected_features_cat)
regressor_features_num = [feat for feat in num_feats if feat in regressor_features]
regressor_features_cat = [feat for feat in cat_feats if feat in regressor_features]
len(regressor_features)

0:	learn: 85845.8162827	total: 86.6ms	remaining: 43.2s
100:	learn: 50486.2589514	total: 2.82s	remaining: 11.1s
200:	learn: 49146.1710582	total: 5.25s	remaining: 7.8s
300:	learn: 48259.8881449	total: 7.66s	remaining: 5.07s
400:	learn: 47561.3748476	total: 10s	remaining: 2.48s
499:	learn: 47007.0267799	total: 14.7s	remaining: 0us


70

In [ ]:
regressor_features

['accum_rur_amt_cm_avg_div_v2',
 'avg_6m_building_services',
 'avg_6m_money_transactions',
 'avg_6m_personal_services',
 'avg_6m_transportation',
 'avg_by_category__amount__sum__cashflowcategory_name__vydacha_nalichnyh_v_bankomate',
 'avg_by_category__amount__sum__cashflowcategory_name__zdorove',
 'avg_cnt_daily_transactions_90d',
 'avg_debet_turn_rur',
 'by_category__amount__sum__eoperation_type_name__vneshnij_perevod_rur',
 'calls_weight_v2',
 'cc_other_rate_max_2avg_prop',
 'channel_chat_s_operatorom_voc_features_full_voc_without_marks_portion',
 'channel_chatbot_voc_features_full_voc_without_marks_portion',
 'channel_ivr_voc_features_36m_voc_not_null_flag',
 'commission_outcome_rur_amt',
 'count_device_samsung_0_60_days',
 'curr_rur_amt_cm_avg',
 'hdb_bki_active_cc_cnt',
 'hdb_bki_active_ip_max_outstand',
 'hdb_bki_active_pil_max_overdue',
 'hdb_bki_total_cc_max_limit',
 'hdb_bki_total_ip_cnt',
 'hdb_bki_total_max_limit',
 'hdb_bki_total_max_overdue_sum',
 'hdb_bki_total_pil_max_li

In [ ]:
# Для перезапуска сессии
# regressor_features = ['accum_rur_amt_cm_avg_div_v2',
#  'avg_6m_building_services',
#  'avg_6m_money_transactions',
#  'avg_6m_personal_services',
#  'avg_6m_transportation',
#  'avg_by_category__amount__sum__cashflowcategory_name__vydacha_nalichnyh_v_bankomate',
#  'avg_by_category__amount__sum__cashflowcategory_name__zdorove',
#  'avg_cnt_daily_transactions_90d',
#  'avg_debet_turn_rur',
#  'by_category__amount__sum__eoperation_type_name__vneshnij_perevod_rur',
#  'cc_other_rate_max_2avg_prop',
#  'channel_chat_s_operatorom_voc_features_full_voc_without_marks_portion',
#  'channel_chatbot_voc_features_full_voc_without_marks_portion',
#  'commission_outcome_rur_amt',
#  'count_device_samsung_0_60_days',
#  'curr_rur_amt_cm_avg',
#  'hdb_bki_active_cc_cnt',
#  'hdb_bki_active_ip_max_outstand',
#  'hdb_bki_active_pil_max_overdue',
#  'hdb_bki_total_cc_max_limit',
#  'hdb_bki_total_ip_cnt',
#  'hdb_bki_total_max_limit',
#  'hdb_bki_total_max_overdue_sum',
#  'hdb_bki_total_pil_max_limit',
#  'hdb_outstand_sum',
#  'incomeValue',
#  'max_cc_largest_max_limit_actoff_30d',
#  'max_pil_largest_max_limit_actoff_90d',
#  'min_cc_max_el_actoff_90d',
#  'min_cc_offer_lifetime_days_actoff_90d',
#  'min_max_limit',
#  'min_pil_max_score_actoff_180d',
#  'min_pil_max_ul_actoff_90d',
#  'mob_cover_days',
#  'percent_outcome_rur_amt',
#  'product_zarplatnaja_karta_voc_features_36m_voc_without_marks_portion',
#  'prof_cc_prof',
#  'profit_income_out_rur_amt_9m',
#  'smsInWavg6m',
#  'summarur_1m_miscellaneous_stores',
#  'summarur_1m_no_cat',
#  'top_channel_weight_v2',
#  'total_rur_amt_cm_avg_div_v2',
#  'total_rur_amt_cm_avg_period_days_ago_v2',
#  'transaction_category_clothers_shoes_sum_amt_d15',
#  'transaction_category_general_store_inc_cnt_2m',
#  'transaction_category_house_repair_inc_cnt_2m',
#  'transaction_category_other_retail_purchase_percent_amt_2m',
#  'transaction_category_supermarket_sum_amt_m3_4',
#  'transaction_category_supermarket_sum_cnt_d15',
#  'turn_cc_cr_max_v2',
#  'turn_cc_db_min_v2',
#  'turn_other_cr_avg_v2',
#  'turn_other_db_max_v2',
#  'uniV5',
#  'voc_features_12m_voc_without_marks_portion',
#  'worksalary_rur_amt',
#  'avg_salary_month',
#  'vpm_2024',
#  'feature_date_year',
#  'feature_date_month',
#  'addrref',
#  'main_last_position_ccode',
#  'oldest_campaignsegment_ccode_for_nss',
#  'oldest_campaignsegment_ccode_for_pil',
#  'segment',
#  'target_bins',
#  'cl0_prob',
#  'cl1_prob',
#  'cl2_prob']

# regressor_features_num = [feat for feat in num_feats if feat in regressor_features]
# regressor_features_cat = [feat for feat in cat_feats if feat in regressor_features]

## Обучение регрессора

In [ ]:
# from copy import deepcopy
# train_df = train_df.sample(frac=1)
df = deepcopy(train_df)
df = pd.concat([df, train_df[train_df['segment'] == 'Affluent']], ignore_index=True)
df = pd.concat([df, train_df[train_df['segment'] == 'Affluent']], ignore_index=True)

df = df.sample(frac=1)

# разделим данные на обучающую, тестовую и валидационную выборки
x_train, x_val, y_train, y_val = train_test_split(df, df['target'], test_size=0.15, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x_train, x_train['target'], test_size=0.15, random_state=42)

In [ ]:
df.shape

(247900, 247)

In [ ]:
train_df.shape

(205962, 264)

In [ ]:
x_train.shape, x_test.shape, x_val.shape

((148806, 241), (26261, 241), (30895, 241))

In [ ]:
# Вариант 1
# regression_model = fit_catboost_regressor(x_train[regressor_features], x_val[regressor_features], y_train, y_val, regressor_features_cat)

0:	learn: 85667.1610963	test: 87157.9537625	best: 87157.9537625 (0)	total: 27.1ms	remaining: 13.5s
100:	learn: 50133.8685830	test: 51717.1565875	best: 51717.1565875 (100)	total: 2.37s	remaining: 9.38s
200:	learn: 48498.6783596	test: 50660.8149555	best: 50660.8149555 (200)	total: 4.8s	remaining: 7.14s
300:	learn: 47375.5459108	test: 50220.4781596	best: 50217.6558571 (298)	total: 7.21s	remaining: 4.77s
400:	learn: 46481.9734806	test: 49950.2015220	best: 49950.2015220 (400)	total: 9.63s	remaining: 2.38s
499:	learn: 45748.8096106	test: 49748.4788857	best: 49748.4788857 (499)	total: 12s	remaining: 0us
bestTest = 49748.47889
bestIteration = 499


In [ ]:
# x_test['target_bins'] = classifier_model.predict(x_test[classifier_features])
# pred_test = regression_model.predict(x_test[regressor_features])
# print(weighted_mean_absolute_error(y_test, pred_test, x_test['w']))

28997.263341114973


Подбор гиперпараметров

In [ ]:
# import optuna

# def objective(trial):

#     # 1. назначаем параметры и значения для перебора
#     params = {
#         "iterations": 2000,
#         "learning_rate": trial.suggest_float("learning_rate", 1e-4, 0.1),
#         "objective": trial.suggest_categorical("objective", ["RMSE"]),
#         "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 3.0, 10.0),
#         "depth": trial.suggest_int("depth", 8, 16),
#         "random_strength": trial.suggest_float("random_strength", 1, 5.0),
#     }

#     # 2. обучаем модель с выбранными параметрами
#     model_cv = fit_catboost_regressor(x_train[regressor_features], x_val[regressor_features], y_train, y_val, regressor_features_cat, params)

#     # 3. делаем предсказание и вовращаем значение метрики
#     preds = model_cv.predict(x_test[regressor_features])
#     wmae = weighted_mean_absolute_error(y_test, preds, x_test['w'])
#     return wmae

In [ ]:
# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=5)

[I 2024-02-18 23:31:50,003] A new study created in memory with name: no-name-10cb4af3-c08a-4771-afd3-bdf0abb02008


0:	learn: 84472.7680238	test: 85956.5813376	best: 85956.5813376 (0)	total: 41ms	remaining: 1m 21s
100:	learn: 48235.1301055	test: 50639.1067467	best: 50639.1067467 (100)	total: 3.39s	remaining: 1m 3s
200:	learn: 46016.1799682	test: 49836.4717240	best: 49836.4717240 (200)	total: 6.81s	remaining: 1m
300:	learn: 44407.7261608	test: 49525.2841302	best: 49525.2841302 (300)	total: 10.2s	remaining: 57.8s
400:	learn: 43127.1650301	test: 49333.2921165	best: 49333.2921165 (400)	total: 13.7s	remaining: 54.5s
500:	learn: 42112.2469286	test: 49211.8612230	best: 49208.5617267 (499)	total: 17.2s	remaining: 51.3s
600:	learn: 41191.7472436	test: 49087.0490641	best: 49084.3939221 (597)	total: 20.6s	remaining: 48s
700:	learn: 40359.2625766	test: 49020.7076833	best: 49018.2234317 (696)	total: 24.1s	remaining: 44.7s
800:	learn: 39591.2125062	test: 48948.3608072	best: 48943.3987027 (787)	total: 27.6s	remaining: 41.4s
900:	learn: 38946.9514499	test: 48876.8928967	best: 48871.4096268 (891)	total: 31.2s	remain

[I 2024-02-18 23:33:02,991] Trial 0 finished with value: 28322.431089372192 and parameters: {'learning_rate': 0.07236920007235434, 'objective': 'RMSE', 'l2_leaf_reg': 8.735090362502403, 'depth': 8, 'random_strength': 2.1885107680488995}. Best is trial 0 with value: 28322.431089372192.


0:	learn: 87349.4165331	test: 88890.7952696	best: 88890.7952696 (0)	total: 213ms	remaining: 7m 5s
100:	learn: 51181.8897939	test: 55917.4065921	best: 55917.4065921 (100)	total: 21.3s	remaining: 6m 41s
200:	learn: 45326.3677010	test: 52600.6946791	best: 52600.6946791 (200)	total: 42s	remaining: 6m 15s
300:	learn: 42621.5035259	test: 51614.4415036	best: 51614.4415036 (300)	total: 1m 2s	remaining: 5m 51s
400:	learn: 40609.6191059	test: 51097.8912758	best: 51097.8912758 (400)	total: 1m 22s	remaining: 5m 28s
500:	learn: 38778.5137080	test: 50811.0806442	best: 50811.0806442 (500)	total: 1m 43s	remaining: 5m 8s
600:	learn: 37160.0225016	test: 50594.2080539	best: 50594.2080539 (600)	total: 2m 3s	remaining: 4m 47s
700:	learn: 35609.4427996	test: 50434.7452212	best: 50434.7452212 (700)	total: 2m 24s	remaining: 4m 27s
800:	learn: 34300.9418840	test: 50309.3994894	best: 50309.3994894 (800)	total: 2m 44s	remaining: 4m 6s
900:	learn: 33224.3597396	test: 50209.5530211	best: 50209.5530211 (900)	total:

[I 2024-02-18 23:40:06,486] Trial 1 finished with value: 29086.336194202748 and parameters: {'learning_rate': 0.01930675187256476, 'objective': 'RMSE', 'l2_leaf_reg': 4.6099541677362925, 'depth': 14, 'random_strength': 2.330044831580755}. Best is trial 0 with value: 28322.431089372192.


...

[I 2024-02-18 23:41:31,256] Trial 2 finished with value: 28427.37445150974 and parameters: {'learning_rate': 0.022987960149238724, 'objective': 'RMSE', 'l2_leaf_reg': 3.4963948440006245, 'depth': 9, 'random_strength': 4.738304627933806}. Best is trial 0 with value: 28322.431089372192.


0:	learn: 84784.4081892	test: 86283.1781387	best: 86283.1781387 (0)	total: 57.8ms	remaining: 1m 55s
100:	learn: 46497.7831222	test: 50745.0187750	best: 50745.0187750 (100)	total: 6.43s	remaining: 2m
200:	learn: 43070.2739176	test: 49976.8244718	best: 49976.8244718 (200)	total: 12.9s	remaining: 1m 55s
300:	learn: 40409.5067956	test: 49556.6995730	best: 49556.6995730 (300)	total: 19.2s	remaining: 1m 48s
400:	learn: 38443.7339619	test: 49353.6409234	best: 49353.6409234 (400)	total: 25.6s	remaining: 1m 42s
500:	learn: 36783.2672023	test: 49257.9427705	best: 49257.9427705 (500)	total: 32s	remaining: 1m 35s
600:	learn: 35451.8621140	test: 49190.5181761	best: 49189.1547762 (589)	total: 38.5s	remaining: 1m 29s
700:	learn: 34201.5796688	test: 49117.2218548	best: 49117.2218548 (700)	total: 44.9s	remaining: 1m 23s
800:	learn: 33101.6257167	test: 49051.0707402	best: 49049.7975693 (797)	total: 51.5s	remaining: 1m 17s
900:	learn: 32109.3481956	test: 49006.9056154	best: 49006.9056154 (900)	total: 58s

[I 2024-02-18 23:42:48,524] Trial 3 finished with value: 28515.607152931363 and parameters: {'learning_rate': 0.06841916908098711, 'objective': 'RMSE', 'l2_leaf_reg': 9.204659616625058, 'depth': 11, 'random_strength': 4.91744135564751}. Best is trial 0 with value: 28322.431089372192.


0:	learn: 86850.5633190	test: 88361.3677528	best: 88361.3677528 (0)	total: 57.8ms	remaining: 1m 55s
100:	learn: 49655.9730704	test: 52376.9845264	best: 52376.9845264 (100)	total: 6.32s	remaining: 1m 58s
200:	learn: 46685.7626764	test: 50862.3704506	best: 50862.3704506 (200)	total: 12.6s	remaining: 1m 53s
300:	learn: 44803.7458355	test: 50387.8269570	best: 50387.3096486 (299)	total: 19.2s	remaining: 1m 48s
400:	learn: 43283.8248956	test: 50063.2200967	best: 50063.2200967 (400)	total: 25.8s	remaining: 1m 42s
500:	learn: 41839.9483370	test: 49798.4559096	best: 49798.2432665 (499)	total: 32.2s	remaining: 1m 36s
600:	learn: 40595.3712753	test: 49576.6965370	best: 49576.6965370 (600)	total: 38.6s	remaining: 1m 29s
700:	learn: 39551.1914695	test: 49455.9538748	best: 49455.9538748 (700)	total: 45s	remaining: 1m 23s
800:	learn: 38581.2170335	test: 49347.8254767	best: 49347.5888830 (798)	total: 51.4s	remaining: 1m 16s
900:	learn: 37710.4947232	test: 49267.6763644	best: 49267.6763644 (900)	total:

[I 2024-02-18 23:45:01,373] Trial 4 finished with value: 28411.337294879053 and parameters: {'learning_rate': 0.02789822563112248, 'objective': 'RMSE', 'l2_leaf_reg': 5.840360903481196, 'depth': 11, 'random_strength': 4.4418765433066785}. Best is trial 0 with value: 28322.431089372192.


In [ ]:
params = {'iterations': 10000, 'learning_rate': 0.1, 'objective': 'RMSE', 'l2_leaf_reg': 5.378491402, 'depth': 9, 'random_strength': 2.9149158}
# params = {'iterations': 4000, 'learning_rate': 0.07236920007235434, 'objective': 'RMSE', 'l2_leaf_reg': 8.735090362502403, 'depth': 8, 'random_strength': 2.1885107680488995}

regression_model = fit_catboost_regressor(x_train[regressor_features], x_val[regressor_features], y_train, y_val, regressor_features_cat, params)
regression_model.save_model("regression_model_3_my.model")

0:	learn: 107110.7828224	test: 107220.6265188	best: 107220.6265188 (0)	total: 71.5ms	remaining: 11m 54s
100:	learn: 58823.3606822	test: 61190.2829326	best: 61190.2829326 (100)	total: 6.15s	remaining: 10m 2s
200:	learn: 52692.8964598	test: 56794.1943311	best: 56794.1943311 (200)	total: 9.84s	remaining: 7m 59s
300:	learn: 48489.7785609	test: 53786.0115306	best: 53786.0115306 (300)	total: 13.5s	remaining: 7m 15s
400:	learn: 45350.0083648	test: 51579.5219404	best: 51579.5219404 (400)	total: 20.7s	remaining: 8m 15s
500:	learn: 42750.7812706	test: 49735.2818099	best: 49735.2818099 (500)	total: 24.3s	remaining: 7m 41s
600:	learn: 40676.3799522	test: 48260.6554435	best: 48260.6554435 (600)	total: 28.1s	remaining: 7m 19s
700:	learn: 39039.3624332	test: 47145.9312674	best: 47145.9312674 (700)	total: 33.9s	remaining: 7m 29s
800:	learn: 37576.4039053	test: 46149.9713832	best: 46149.9713832 (800)	total: 40.1s	remaining: 7m 40s
900:	learn: 36326.5777810	test: 45329.2892925	best: 45329.2892925 (900)	

In [ ]:
x_test['target_bins'] = classifier_model.predict(x_test[classifier_features])
pred_test = regression_model.predict(x_test[regressor_features])
print(weighted_mean_absolute_error(y_test, pred_test, x_test['w']))

20140.937491308825


In [ ]:
feature_importance = pd.Series(regression_model.get_feature_importance(), x_train[regressor_features].columns)

In [ ]:
feature_importance

accum_rur_amt_cm_avg_div_v2     0.379232
avg_6m_building_services        0.651015
avg_6m_money_transactions       3.468012
avg_6m_personal_services        0.569593
avg_6m_transportation           0.817490
                                 ...    
segment                         5.159063
target_bins                    14.774268
cl0_prob                        0.996989
cl1_prob                        4.485657
cl2_prob                        2.439049
Length: 68, dtype: float64

## Использование обученного регрессора

In [ ]:
# regression_model = CatBoostRegressor()
# regression_model.load_model('best_models/regression_model.model')
# regressor_features = regression_model.feature_names_
# regressor_features_cat = [feat for feat in cat_feats if feat in regressor_features]

In [ ]:
# x_test['target_bins'] = classifier_model.predict(x_test[classifier_features])
# pred_test = regression_model.predict(x_test[regressor_features])
# print(weighted_mean_absolute_error(y_test, pred_test, x_test['w']))

22587.44454157929


# Инференс

In [ ]:
# test_df['target_bins'] = classifier_model.predict(test_df[classifier_features])
test_df['predict'] = regression_model.predict(test_df[regressor_features])
test_df[['client_id','predict']].set_index('client_id').to_csv("commit_only_me_feats_part_train.csv", sep=",", decimal=".")

Обучение на всем

In [ ]:
params = {"iterations": 4000, 'learning_rate': 0.04, 'l2_leaf_reg': 2.911507368, 'depth': 7, 'random_strength': 0.49193290791022337}
classifier_model = fit_catboost_classifier(train_df[classifier_features], None, train_df['target_bins'], None, classifier_features_cat, params)
classifier_model.save_model("classifier_model_all_only_my_feats.model")

0:	learn: 1.0781660	total: 30ms	remaining: 2m
100:	learn: 0.7235997	total: 1.82s	remaining: 1m 10s
200:	learn: 0.6860338	total: 3.43s	remaining: 1m 4s
300:	learn: 0.6694267	total: 5.82s	remaining: 1m 11s
400:	learn: 0.6594136	total: 9.37s	remaining: 1m 24s
500:	learn: 0.6517269	total: 11s	remaining: 1m 16s
600:	learn: 0.6454547	total: 12.5s	remaining: 1m 10s
700:	learn: 0.6400352	total: 14.2s	remaining: 1m 6s
800:	learn: 0.6353191	total: 15.7s	remaining: 1m 2s
900:	learn: 0.6311339	total: 17.3s	remaining: 59.5s
1000:	learn: 0.6270643	total: 20.7s	remaining: 1m 1s
1100:	learn: 0.6231511	total: 23s	remaining: 1m
1200:	learn: 0.6196150	total: 24.5s	remaining: 57.2s
1300:	learn: 0.6163049	total: 26.1s	remaining: 54.2s
1400:	learn: 0.6128903	total: 27.7s	remaining: 51.4s
1500:	learn: 0.6096339	total: 29.3s	remaining: 48.8s
1600:	learn: 0.6063054	total: 30.9s	remaining: 46.3s
1700:	learn: 0.6031652	total: 35.2s	remaining: 47.5s
1800:	learn: 0.6001406	total: 36.8s	remaining: 44.9s
1900:	learn

In [ ]:
params = {'iterations': 10000, 'learning_rate': 0.1, 'objective': 'RMSE', 'l2_leaf_reg': 5.378491402, 'depth': 9, 'random_strength': 2.9149158}
# params = {'iterations': 4000, 'learning_rate': 0.07236920007235434, 'objective': 'RMSE', 'l2_leaf_reg': 8.735090362502403, 'depth': 8, 'random_strength': 2.1885107680488995}

regression_model = fit_catboost_regressor(train_df[regressor_features], None, train_df['target'], None, regressor_features_cat, params)
regression_model.save_model("regression_model_all_only_my_feats.model")

0:	learn: 83352.6176832	total: 70.2ms	remaining: 11m 42s
100:	learn: 47341.4727933	total: 6.43s	remaining: 10m 30s
200:	learn: 44471.1665517	total: 10.3s	remaining: 8m 23s
300:	learn: 42461.3463552	total: 14.3s	remaining: 7m 40s
400:	learn: 40837.6863256	total: 20.2s	remaining: 8m 3s
500:	learn: 39453.7746590	total: 24.2s	remaining: 7m 39s
600:	learn: 38347.4758541	total: 28.2s	remaining: 7m 21s
700:	learn: 37322.6789079	total: 34.2s	remaining: 7m 34s
800:	learn: 36408.9316877	total: 38.3s	remaining: 7m 19s
900:	learn: 35561.1911536	total: 42.8s	remaining: 7m 12s
1000:	learn: 34808.3186130	total: 48.4s	remaining: 7m 14s
1100:	learn: 34146.3732422	total: 52.4s	remaining: 7m 3s
1200:	learn: 33493.8785230	total: 58.6s	remaining: 7m 9s
1300:	learn: 32850.6611421	total: 1m 2s	remaining: 6m 59s
1400:	learn: 32274.9646877	total: 1m 6s	remaining: 6m 50s
1500:	learn: 31704.6765233	total: 1m 12s	remaining: 6m 51s
1600:	learn: 31154.0436073	total: 1m 16s	remaining: 6m 43s
1700:	learn: 30639.27468

In [ ]:
test_df['target_bins'] = classifier_model.predict(test_df[classifier_features])
test_df['predict'] = regression_model.predict(test_df[regressor_features])
test_df[['client_id','predict']].set_index('client_id').to_csv("commit_all_only_my_feats.csv", sep=",", decimal=".")